In [1]:
import pandas as pd
import io
from operator import itemgetter

In [85]:
data = pd.read_csv("./Output/MissingMigrants.csv")
display(data[32:55])

,Region of Incident,Reported Year,Reported Month,Number Dead,Minimum Estimated Number of Missing,Total Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Cause of Death,Location Description,Location Coordinates,Migration Route
32,Mediterranean,2019,Feb,2.0,4.0,6,Unknown,1.0,1.0,Unknown,Drowning,Morocco,"35.856284991833, -5.854248522049",Western Mediterranean
33,US-Mexico Border,2019,Feb,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Unknown,"Arizona, USA","32.057701900000, -112.143021500000",Unknown
34,US-Mexico Border,2019,Feb,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Unknown,"Arizona, USA","32.057701900000, -112.143021500000",Unknown
35,US-Mexico Border,2019,Feb,1.0,Unknown,1,Unknown,Unknown,1.0,Unknown,Unknown,"Arizona, USA","32.291122000000, -111.377669100000",Unknown
36,Mediterranean,2019,Feb,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Drowning,Libya,"32.799680261076, 12.740310887952",Central Mediterranean
37,US-Mexico Border,2019,Feb,1.0,Unknown,1,Unknown,Unknown,1.0,Unknown,Drowning,"Texas, USA","28.620958685162, -100.160775037960",Unknown
38,US-Mexico Border,2019,Feb,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Unknown,"Arizona, USA","31.802384100000, -110.262284200000",Unknown
39,US-Mexico Border,2019,Feb,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Unknown,"Arizona, USA","32.331803200000, -112.524509300000",Unknown
40,Mediterranean,2019,Feb,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Drowning,Libya,"32.795883457110, 12.734512549273",Central Mediterranean
41,North Africa,2019,Feb,2.0,Unknown,2,Unknown,2.0,Unknown,Unknown,Unknown,Morocco,"35.143072914540, -2.987014300369",Unknown


In [3]:
data.dtypes

Region of Incident                     object
Reported Year                           int64
Reported Month                         object
Number Dead                            object
Minimum Estimated Number of Missing    object
Total Dead and Missing                  int64
Number of Survivors                    object
Number of Females                      object
Number of Males                        object
Number of Children                     object
Cause of Death                         object
Location Description                   object
Location Coordinates                   object
Migration Route                        object
dtype: object

In [4]:
import requests
BASE_URL = "https://api.darksky.net/forecast"

In [5]:
# Github API Authentication: https://developer.github.com/v3/auth/
import os
from dotenv import load_dotenv
load_dotenv()

if not "TOKEN" in os.environ:
    raise ValueError("You should pass a GITHUB TOKEN, see: https://github.com/settings/tokens")
TOKEN = os.environ["TOKEN"]

In [6]:
type(data['Location Coordinates'][0])

str

In [47]:
def get_lat_lon(st): 
    
    try: 
        st = st.replace('[', '').replace(']', '') # fuera corchetes
        lat, lon = st.split(',')
        res = {'latitud': float(lat.strip()), 'longitud': float(lon.strip())}
    except: 
        res = {'latitud': 0, 'longitud': 0}
    return res

def get_lat(st): 
    return get_lat_lon(st).get('latitud')

def get_lon(st): 
    return get_lat_lon(st).get('longitud')

data['latitud'] = data['Location Coordinates'].apply(get_lat)

data['longitud'] = data['Location Coordinates'].apply(get_lon)

data.head()

,Region of Incident,Reported Year,Reported Month,Number Dead,Minimum Estimated Number of Missing,Total Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Cause of Death,Location Description,Location Coordinates,Migration Route,latitud,longitud
0,Mediterranean,2019,Mar,Unknown,2.0,2,36.0,Unknown,2.0,Unknown,Drowning,Greece,"38.362368696592, 26.172509473654",Eastern Mediterranean,38.362369,26.172509
1,Mediterranean,2019,Mar,4.0,Unknown,4,11.0,3.0,Unknown,1.0,Drowning,Turkey,"39.441975591614, 26.378816195919",Eastern Mediterranean,39.441976,26.378816
2,Mediterranean,2019,Mar,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Drowning,Spain,"35.871901875921, -5.343037665842",Western Mediterranean,35.871902,-5.343038
3,Mediterranean,2019,Mar,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Drowning,Morocco,"35.635115912988, -5.275650103548",Western Mediterranean,35.635116,-5.275650
4,Central America,2019,Mar,1.0,Unknown,1,Unknown,Unknown,1.0,Unknown,Accident,Mexico,"19.334475177429, -98.069823987538",Unknown,19.334475,-98.069824


In [8]:
data.dtypes

Region of Incident                      object
Reported Year                            int64
Reported Month                          object
Number Dead                             object
Minimum Estimated Number of Missing     object
Total Dead and Missing                   int64
Number of Survivors                     object
Number of Females                       object
Number of Males                         object
Number of Children                      object
Cause of Death                          object
Location Description                    object
Location Coordinates                    object
Migration Route                         object
latitud                                float64
longitud                               float64
dtype: object

In [9]:
def get_3_items(data, ind_row): 
    latitud = data.loc[ind_row, 'latitud']
    longitud = data.loc[ind_row, 'longitud']
    return (latitud, longitud)

get_3_items(data, 0) 

(38.362368696592, 26.172509473654)

In [10]:
lista_marzo=[]
for ind in range(31): 
    lat, lon = get_3_items(data, ind)
    res = requests.get("{}/{}/{},{},1551398400?exclude=currently,flags, minutely, hourly".format(BASE_URL, TOKEN, lat, lon))
    temperatureMax = res.json().get('daily').get('data')[0].get('temperatureMax')
    temperatureMin = res.json().get('daily').get('data')[0].get('temperatureMin')
    uvIndex = res.json().get('daily').get('data')[0].get('uvIndex')
    diccion ={}
    diccion.update({"temperatureMax": temperatureMax, "temperatureMin": temperatureMin, "uvIndex": uvIndex})
    lista_marzo.append(diccion)

In [11]:
data_marzo_2019= pd.DataFrame(lista_marzo)

In [12]:
lista_febrero=[]
for ind in range(32,76): 
    lat, lon = get_3_items(data, ind)
    res = requests.get("{}/{}/{},{},1548979200?exclude=currently,flags, minutely, hourly".format(BASE_URL, TOKEN, lat, lon))

In [13]:
#lista_febrero=[]
#for ind in range(32,76): 
    #lat, lon = get_3_items(data, ind)
    #res = requests.get("{}/{}/{},{},1548979200?exclude=currently,flags, minutely, hourly".format(BASE_URL, TOKEN, lat, lon))
    #temperatureMax = res.json().get('daily').get('data')[0].get('temperatureMax')
    #temperatureMin = res.json().get('daily').get('data')[0].get('temperatureMin')
    #uvIndex = res.json().get('daily').get('data')[0].get('uvIndex')
    #diccion ={}
    #diccion.update({"temperatureMax": temperatureMax, "temperatureMin": temperatureMin, "uvIndex": uvIndex})
    #lista_febrero.append(diccion)

In [14]:
data_febrero_2019 = pd.DataFrame(lista_febrero)
display(data_febrero_2019)

,temperatureMax,temperatureMin,uvIndex
0,62.15,51.44,3
1,67.96,42.20,4
2,67.96,42.20,4
3,73.51,45.91,4
4,75.19,55.91,4
5,62.23,48.39,3
6,68.02,34.73,5
7,69.65,43.71,4
8,75.67,55.55,4
9,70.56,54.01,4


In [15]:
lista_enero=[]
for ind in range(77,132): 
    lat, lon = get_3_items(data, ind)
    res = requests.get("{}/{}/{},{},1546300800?exclude=currently,flags, minutely, hourly".format(BASE_URL, TOKEN, lat, lon))
    temperatureMax = res.json().get('daily').get('data')[0].get('temperatureMax')
    temperatureMin = res.json().get('daily').get('data')[0].get('temperatureMin')
    uvIndex = res.json().get('daily').get('data')[0].get('uvIndex')
    diccion ={}
    diccion.update({"temperatureMax": temperatureMax, "temperatureMin": temperatureMin, "uvIndex": uvIndex})
    lista_enero.append(diccion)

In [16]:
data_enero_2019 = pd.DataFrame(lista_enero)
display(data_enero_2019)

,temperatureMax,temperatureMin,uvIndex
0,84.27,73.79,9
1,51.35,31.08,3
2,51.35,31.08,3
3,51.35,31.08,3
4,85.51,65.39,6
5,85.48,80.13,9
6,61.79,39.94,5
7,65.59,43.52,5
8,60.62,48.56,6
9,45.10,29.41,4


In [39]:
lista_diciembre_2018=[]
for ind in range(133,185): 
    lat, lon = get_3_items(data, ind)
    res = requests.get("{}/{}/{},{},1543622400?exclude=currently,flags, minutely, hourly".format(BASE_URL, TOKEN, lat, lon))
    temperatureMax = res.json().get('daily').get('data')[0].get('temperatureMax')
    temperatureMin = res.json().get('daily').get('data')[0].get('temperatureMin')
    uvIndex = res.json().get('daily').get('data')[0].get('uvIndex')
    diccion ={}
    diccion.update({"temperatureMax": temperatureMax, "temperatureMin": temperatureMin, "uvIndex": uvIndex})
    lista_diciembre_2018.append(diccion)

In [40]:
lista_diciembre_2018 = pd.DataFrame(lista_diciembre_2018)
display(lista_diciembre_2018)

,temperatureMax,temperatureMin,uvIndex
0,64.23,45.84,4
1,68.21,64.84,3
2,63.72,60.02,3
3,87.02,64.64,5
4,49.41,40.17,1
5,82.77,51.24,5
6,65.00,58.41,3
7,63.55,48.23,3
8,59.73,42.04,2
9,63.57,53.91,2


In [48]:
lista_noviembre_2018=[]
for ind in range(186,259): 
    lat, lon = get_3_items(data, ind)
    res = requests.get("{}/{}/{},{},1543622400?exclude=currently,flags, minutely, hourly".format(BASE_URL, TOKEN, lat, lon))
    temperatureMax = res.json().get('daily').get('data')[0].get('temperatureMax')
    temperatureMin = res.json().get('daily').get('data')[0].get('temperatureMin')
    uvIndex = res.json().get('daily').get('data')[0].get('uvIndex')
    diccion ={}
    diccion.update({"temperatureMax": temperatureMax, "temperatureMin": temperatureMin, "uvIndex": uvIndex})
    lista_noviembre_2018.append(diccion)

In [50]:
data_noviembre_2018 =pd.DataFrame(lista_noviembre_2018)
display(data_noviembre_2018)

,temperatureMax,temperatureMin,uvIndex
0,63.33,57.34,3
1,62.59,59.09,2
2,63.67,61.30,2
3,83.69,64.19,5
4,50.21,43.46,3
5,84.61,65.40,5
6,46.68,40.79,3
7,43.84,29.37,2
8,63.93,62.27,2
9,73.06,51.75,5


In [51]:
dataframes_temperature = [data_marzo_2019, data_febrero_2019, data_enero_2019, lista_diciembre_2018, data_noviembre_2018]

In [52]:
dataframe_merge = pd.concat(dataframes_temperature)

In [163]:
display(dataframe_merge)

,temperatureMax,temperatureMin,uvIndex
0,62.14,37.53,5
1,62.16,48.50,5
2,66.90,57.19,4
3,71.45,54.19,5
4,74.11,41.19,13
5,74.57,37.74,9
6,74.57,37.74,9
7,99.22,72.77,10
8,74.57,37.74,9
9,74.48,37.75,9


In [170]:
data_filter = data.iloc[0:255]
#display(data_filter)

data_filter.reset_index(drop=True, inplace=True)
dataframe_merge.reset_index(drop=True, inplace=True)

data_analysis = pd.concat( [data_filter, dataframe_merge], axis=1) 
display(data_analysis)


,Region of Incident,Reported Year,Reported Month,Number Dead,Minimum Estimated Number of Missing,Total Dead and Missing,Number of Survivors,Number of Females,Number of Males,Number of Children,Cause of Death,Location Description,Location Coordinates,Migration Route,temperatureMax,temperatureMin,uvIndex
0,Mediterranean,2019,Mar,Unknown,2.0,2,36.0,Unknown,2.0,Unknown,Drowning,Greece,"38.362368696592, 26.172509473654",Eastern Mediterranean,62.14,37.53,5
1,Mediterranean,2019,Mar,4.0,Unknown,4,11.0,3.0,Unknown,1.0,Drowning,Turkey,"39.441975591614, 26.378816195919",Eastern Mediterranean,62.16,48.50,5
2,Mediterranean,2019,Mar,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Drowning,Spain,"35.871901875921, -5.343037665842",Western Mediterranean,66.90,57.19,4
3,Mediterranean,2019,Mar,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Drowning,Morocco,"35.635115912988, -5.275650103548",Western Mediterranean,71.45,54.19,5
4,Central America,2019,Mar,1.0,Unknown,1,Unknown,Unknown,1.0,Unknown,Accident,Mexico,"19.334475177429, -98.069823987538",Unknown,74.11,41.19,13
5,US-Mexico Border,2019,Mar,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Unknown,"Arizona, USA","32.057499000000, -111.666072500000",Unknown,74.57,37.74,9
6,US-Mexico Border,2019,Mar,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Unknown,"Arizona, USA","32.057499000000, -111.666072500000",Unknown,74.57,37.74,9
7,Southeast Asia,2019,Mar,8.0,2.0,10,Unknown,1.0,Unknown,Unknown,Accident,Myanmar,"13.865704814542, 99.587248723929",Unknown,99.22,72.77,10
8,US-Mexico Border,2019,Mar,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Unknown,"Arizona, USA","32.057499000000, -111.666072500000",Unknown,74.57,37.74,9
9,US-Mexico Border,2019,Mar,1.0,Unknown,1,Unknown,Unknown,Unknown,Unknown,Unknown,"Arizona, USA","32.057499000000, -111.666072500000",Unknown,74.48,37.75,9


In [171]:
data_analysis.to_csv('./Output/MissingMigrants_final.csv', index=False)